In [19]:
import pandas as pd

In [20]:
data = pd.read_csv('/content/dados.csv')

In [21]:
data.head()

,titulo,autor,ISBN_13,ISBN_10,ano,paginas,idioma,editora,rating,avaliacao,resenha,abandonos,relendo,querem_ler,lendo,leram,descricao,genero,male,female
0,Orçamento sem falhas,Nath Finanças,9.786556e+12,6555601566,2021,128,português,Intrínseca,0.0,0,0,0,0,1,0,0,"Quando o assunto é dinheiro, não dá para achar...","Economia, Finanças / Literatura Brasileira / N...",28,72
1,Minha Sombria Vanessa,Kate Elizabeth Russell,9.788551e+12,8551006274,2020,432,português,Intrínseca,4.3,2,653,48,3,6,389,2,Elogiado por Gillian Flynn e considerado um do...,Drama / Ficção / Literatura Estrangeira / Susp...,7,93
2,Recursão,Blake Crouch,9.788551e+12,8551005375,2020,320,português,Intrínseca,4.4,2,618,22,2,6,295,2,E se um dia memórias vívidas de coisas que nun...,Ficção / Ficção científica / Literatura Estran...,21,79
3,"M, o Filho do Século",Antonio Scurati,9.788551e+12,855100607X,2020,816,português,Intrínseca,4.6,60,12,1,0,951,49,55,"O romance M, o Filho do Século conta em trama ...","Biografia, Autobiografia, Memórias / História ...",37,63
4,Oblivion Song: Entre Dois Mundos,Robert Kirkman,9.788551e+12,8551006258,2020,136,português,Intrínseca,4.1,166,16,0,0,376,6,184,Mestre em traçar universos distópicos permeado...,"Ficção / HQ, comics, mangá / Literatura Estran...",44,56


In [22]:
data.isnull().sum()

,0
titulo,0
autor,4
ISBN_13,759
ISBN_10,759
ano,0
paginas,0
idioma,2
editora,1
rating,0
avaliacao,0


In [23]:
# Limpeza dos dados
data.drop(['ISBN_13','ISBN_10'],axis=1, inplace=True)
data = data[data['descricao'].notna()]

In [24]:
data.isnull().sum()

,0
titulo,0
autor,2
ano,0
paginas,0
idioma,1
editora,1
rating,0
avaliacao,0
resenha,0
abandonos,0


In [25]:
# Pré-processamento das colunas de descrição

import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Definir uma função para limpar o texto
def clean_text(text):
    # Remover números e pontuação
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Converter para minúsculas
    text = text.lower()
    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar a função de limpeza nas descrições
data['descricao'] = data['descricao'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Gerar a matriz TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Obter a lista de stop words em português
stop_words_pt = stopwords.words('portuguese')

# Criar o TfidfVectorizer com as stop words em português
tfidf = TfidfVectorizer(stop_words=stop_words_pt) # Passing the list of Portuguese stopwords

tfidf_matrix = tfidf.fit_transform(data['descricao'])

In [28]:
# Calcular a similaridade de cosseno entre os livros
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)

In [29]:
# Função para recomendar livros similares (baseado em conteúdo)
def recomendar_livros_similares(book_index, cosine_sim=cosine_sim):
    sim_scores = list(enumerate(cosine_sim[book_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    return sim_scores[1:6]  # Recomendando os 5 livros mais similares

In [30]:
# Filtragem colaborativa (SVD)
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Criar 'usuario_id' fictício
data['usuario_id'] = data.index

# Definir o leitor e carregar o conjunto de dados de avaliações
reader = Reader(rating_scale=(0, 5))
data_svd = Dataset.load_from_df(data[['usuario_id', 'titulo', 'rating']], reader)

# Dividir o conjunto de dados em treino e teste
trainset, testset = train_test_split(data_svd, test_size=0.2)

# Treinar o modelo SVD
model_svd = SVD()
model_svd.fit(trainset)

In [35]:
# Supondo que 'data' tenha a coluna 'titulo' que contém os nomes dos livros

# Criar um dicionário que mapeia os IDs dos livros para os títulos
id_to_title = pd.Series(data['titulo'].values, index=data.index).to_dict()

# Gerar previsões apenas para 10 itens não avaliados
usuario_id = 1  # ID do usuário
anti_testset = [x for x in trainset.all_ratings() if x[0] == usuario_id]
all_predictions = model_svd.test(anti_testset)

# Ordenar as previsões e pegar os 10 melhores
top_n = sorted(all_predictions, key=lambda x: x.est, reverse=True)[:10]
livros_colaborativos = [(pred[0], pred.est) for pred in top_n]  # Pegar os IDs e ratings estimados

# Obter os títulos dos livros recomendados (colaborativos)
livros_colaborativos_titulos = [(id_to_title[l[0]], l[1]) for l in livros_colaborativos]

# Obter os livros semelhantes com os títulos (baseado em conteúdo)
livros_similares_titulos = [(id_to_title[l[0]], l[1]) for l in livros_similares]

# Exibir as recomendações finais
print("Recomendações finais (híbridas):")

# Exibir as recomendações colaborativas com os títulos
for livro, rating in livros_colaborativos_titulos:
    print(f"Livro: {livro}, Rating estimado: {rating}")

# Exibir livros semelhantes com títulos
print("\nLivros semelhantes (baseado em conteúdo):")
for livro_similar, rating_similar in livros_similares_titulos:
    print(f"Livro semelhante: {livro_similar}, Similaridade: {rating_similar}")


Recomendações finais (híbridas):
Livro: Minha Sombria Vanessa, Rating estimado: 3.894975048684834

Livros semelhantes (baseado em conteúdo):
Livro semelhante: Vem Comigo, Similaridade: 0.21346509647558845
Livro semelhante: Brandjam, Similaridade: 0.17584838987551105
Livro semelhante: A Menina e o Pássaro Encantado, Similaridade: 0.17096264530781677
Livro semelhante: O ponto de equilíbrio, Similaridade: 0.1658557135135587
Livro semelhante: Um trabalho para amar, Similaridade: 0.1645334777870527
